In [1]:
import numpy as np
import numpy.polynomial as P
import scipy as sp
from matplotlib import pyplot as plt
from tqdm import tqdm
import Algo2
import Algo1
import ULA
from sklearn.preprocessing import PolynomialFeatures
from joblib import Parallel, delayed
import ZVnbrosse

In [2]:
import importlib
importlib.reload(Algo2)

<module 'Algo2' from '/Users/sergosamsonoff/Research/VR-MCMC/Code/Algo2.py'>

In [3]:
def f_grad(x):
    a = 1 / np.sqrt(2)
    return x-a+2*a/(1 + np.exp(2* (x * a)))

def f(x):
    a = 1 / np.sqrt(2)
    return 1/2 * (x-a)**2 - np.log(1 + np.exp(-2 * x * a))

def set_func(x):
    #function of interest to compute averages
    return x

Approximation results: use $Q_{l-p}$ to approximate family of $Q_{l,p}$ relying on approximate stationarity of the chain.

In [4]:
#generate samples from mixture of normals
N_burn = 10000
N_train = 1000
gamma = 0.1
N_traj_train = 1
X_train = np.zeros((N_traj_train,N_train),dtype = float)

for j in range(N_traj_train):
    np.random.seed(42+j)
    x0 = np.random.randn()
    x_cur = x0
    #burn-in
    for i in range(N_burn):
        x_cur = x_cur - gamma*f_grad(x_cur) + np.sqrt(2*gamma)*np.random.randn()
    #training sample
    for i in range(N_train):
        X_train[j,i] = x_cur
        x_cur = x_cur - gamma*f_grad(x_cur) + np.sqrt(2*gamma)*np.random.randn()
X_last = X_train[0,-1]

Optimize coefficients

In [5]:
#degree
max_deg = 5
#lag order
lag = 20
#polynomial coefficients
coefs_poly = np.zeros((lag,max_deg+1),dtype = float)

for i in range(lag):
    y = np.array([])
    x = np.array([])
    for j in range(N_traj_train):
        y_cur = set_func(X_train[j,i+1:])
        y = np.concatenate([y,y_cur])
        x_cur = X_train[j,:-(i+1)]
        x = np.concatenate([x,x_cur])
    res = np.polynomial.polynomial.polyfit(x,y,max_deg)
    coefs_poly[i,:] = res

In [6]:
sp.special.factorial2(21, exact=False)
moments_stand_norm = np.zeros(2*max_deg+1,dtype = float)
for i in range(len(moments_stand_norm)):
    moments_stand_norm[i] = sp.special.factorial2(i+1, exact=False)
#eliminate odd
moments_stand_norm[1::2] = 0
print(moments_stand_norm)

[1.0000e+00 0.0000e+00 3.0000e+00 0.0000e+00 1.5000e+01 0.0000e+00
 1.0500e+02 0.0000e+00 9.4500e+02 0.0000e+00 1.0395e+04]


In [7]:
N_burn = 10000
N_test = 1000 #size of test part
N_min = 100 #minimal number of observations to compute \pi_N
gamma = 0.1
X_test = np.zeros(N_test,dtype = float)
Noise = np.zeros_like(X_test)
N_traj = 10

test_stat_vanilla = np.zeros((N_traj,N_test),dtype = float)
test_stat_vr = np.zeros((N_traj,N_test),dtype = float)

In [8]:
for ind in range(N_traj):
    np.random.seed(1453 + ind)
    #x0 = np.random.randn()
    #x_cur = x0
    #burn-in
    #for i in range(N_burn):
        #x_cur = x_cur - gamma*f_grad(x_cur) + np.sqrt(2*gamma)*np.random.randn()
    #training sample
    x_cur = X_last
    for i in range(N_test):
        Noise[i] = np.random.randn()
        x_cur = x_cur - gamma*f_grad(x_cur) + np.sqrt(2*gamma)*Noise[i]
        X_test[i] = x_cur
    #compute polynomials at Z_l
    poly_vals = np.zeros((max_deg+1,N_test),dtype = float)
    for k in range(max_deg+1):
        c = np.zeros(max_deg+1)
        c[k] = 1
        poly_vals[k,:] = P.hermite_e.hermeval(Noise,c)/np.sqrt((np.sqrt(2*np.pi)*sp.special.factorial(k)))
    f_vals_vanilla = set_func(X_test)
    cvfs = np.zeros_like(f_vals_vanilla)
    for i in range(100,len(cvfs)):
        #start computing a_{p-l} coefficients
        num_poly = min(lag,i)
        a_vals = np.zeros((num_poly,max_deg+1),dtype = float)
        for npol in range(num_poly):#for a fixed lag Q function
            #compute \hat{a} with fixed lag
            x = X_test[i-2-npol]
            a_cur = np.zeros(max_deg+1,dtype=float)
            for m in range(len(coefs_poly[0])):
                poly_vspom = np.zeros(max_deg+1,dtype=float)
                for u in range(m+1):
                    poly_vspom[u] = ((x-gamma*f_grad(x))**(m-u))*((np.sqrt(2*gamma))**u)*sp.special.binom(m,u)
                a_cur = P.polynomial.polyadd(a_cur,coefs_poly[npol,m]*poly_vspom) 
            for k in range(max_deg+1):
                c = np.zeros(max_deg+1)
                c[k] = 1
                herm_coef = P.hermite_e.herme2poly(c)
                #normalize now
                herm_coef = herm_coef / np.sqrt((np.sqrt(2*np.pi)*sp.special.factorial(k)))
                integr_coefs = P.polynomial.polymul(herm_coef,a_cur)
                a_vals[npol,k] = np.dot(integr_coefs,moments_stand_norm[:len(integr_coefs)])
            #OK, now I have coefficients of the polynomial, and I need to integrate it w.r.t. Gaussian measure
        cvfs[i] = np.sum(a_vals*(poly_vals[:,i-num_poly:i].T))
        #cvfs[i] = np.sum(np.mean(a_vals,axis = 0))
        if (i%100 == 0):
            print("100 observations proceeded")
        #save results
        test_stat_vanilla[ind,i] = np.mean(f_vals_vanilla[:i])
        test_stat_vr[ind,i] = test_stat_vanilla[ind,i] - cvfs[i]/i

100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
100 observations proceeded
1

In [11]:
print(test_stat_vanilla[1,N_min:])
print(test_stat_vr[1,N_min:])

[1.65598289 1.65251426 1.65070595 1.64613828 1.64753606 1.6505027
 1.64651145 1.64322347 1.63886391 1.63502119 1.63851211 1.64360281
 1.6439386  1.64064989 1.63716636 1.63117952 1.62828156 1.62490718
 1.62469658 1.61864709 1.60419456 1.5871959  1.57185665 1.55539246
 1.54393829 1.53544292 1.53244896 1.52654861 1.52145883 1.51384827
 1.49535273 1.47608481 1.45641101 1.43627216 1.41745355 1.40003507
 1.38670446 1.37396639 1.36231121 1.3504072  1.3417304  1.33473354
 1.32979215 1.32298894 1.31708715 1.31147882 1.30482002 1.30096168
 1.29606769 1.29254181 1.28712562 1.28064792 1.27826199 1.27788385
 1.27819465 1.28281468 1.28724118 1.28903972 1.29258912 1.29462017
 1.29494391 1.28955863 1.2845964  1.27558831 1.27417531 1.2695974
 1.25847945 1.24697947 1.2396776  1.23384999 1.22777313 1.21893815
 1.20913485 1.19893316 1.18608582 1.17028454 1.1511339  1.12974125
 1.10555627 1.07949931 1.05539153 1.03462058 1.01396495 0.99593445
 0.98046308 0.96963644 0.95802327 0.9448823  0.93385085 0.924853

In [12]:
vars_vanilla = np.var(test_stat_vanilla,axis = 0)
vars_adj = np.var(test_stat_vr,axis = 0)
print(vars_vanilla[N_min:N_min+10])
print(vars_adj[N_min:N_min+10])
print(np.mean(vars_adj[N_min:]/vars_vanilla[N_min:]))

[0.2455638  0.24665074 0.24830918 0.2487423  0.25096556 0.25349112
 0.25439903 0.25448522 0.25389267 0.25285693]
[0.21349191 0.21511742 0.21723522 0.21189183 0.21257877 0.21091325
 0.20844613 0.21417611 0.21349367 0.21897067]
0.9815929289766613
